In [1]:
# Define Cloudberry configuration

import cloudberry.api as cb
cb_port = 9000
cb_config = cb.CloudberryConfig(f'http://localhost:{cb_port}')

In [2]:
# Initialize API for data upload

cb_uploader = cb.AgeFileUploader(cb_config)

files_paths = [
    "./data/emas-20190411T232808.log",
    "./data/emas-20190411T234810.log",
    "./data/emas-20190412T000813.log",
    "./data/emas-20190412T120536.log",
]

files_details = cb.AgeUploadDetails(
    headers_keys={"[WH]": "[W]", "[SH]": "[S]", "[BH]": "[B]"}
)

experiment_name = 'Example AgE experiment'

uploaded_computations = []
for fp in files_paths:
    computation = cb_uploader.upload_file(file_path=fp, experiment_name=experiment_name, details=files_details)
    uploaded_computations.append(computation)

In [3]:
# Compare uploaded computations and compute average and std

cb_computations = cb.ComputationSeries(cb_config)

field = 'BEST_SOLUTION_SO_FAR'
series_pack = cb_computations.for_computations(uploaded_computations, field)

In [4]:
# Preview returned series as data frames

# series_pack.series[0].as_data_frame
# series_pack.average_series.as_data_frame

In [5]:
# Perform conversion to plots model

from cloudberry.converters import SeriesConverter

plot_pack = SeriesConverter.data_series_pack_to_plot_series_pack(
    ds=series_pack,
    x_field="_time",
    y_field=field,
    y_err_field="stddev"
)

In [9]:
# Configure cloudberry.plots

import cloudberry.plots as cp

props = cp.PlotProperties.default()
props.title = "Best fitness for uploaded computations"
props.show_title = True
props.x_axis_name = "Time in milliseconds"
props.y_axis_name = "Best fitness"
# props.show_series = False
# props.show_error_bars = True
props.default_series_kind = cp.PlotSeriesKind.LINE

builder = cp.PlotBuilder(props)
builder.add_pack(plot_pack)
plot = builder.plot()
plot

In [7]:
# Optionally export plot to file
cp.PlotlyExporter(plot).write_image(file='best_solution_computations.png', scale=5)

In [10]:
# Cleanup
cb_deletion = cb.Deletion(cb_config)
cb_deletion.delete_computations(uploaded_computations)

# Verify cleanup
cb_meta = cb.Metadata(cb_config)
cb_meta.experiment_computation_api().find_all()